# Model Training

In [1]:
import sys
import warnings
import pandas as pd
import numpy as np
import joblib
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import clone

sys.path.append("../../")

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

C:\Users\ralph\regression-model\venv\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


## Load Data

In [2]:
df = pd.read_csv("../../data/insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Create Training and Test Sets

In [3]:
mask = np.random.rand(len(df)) < 0.8

training_set = df[mask]

testing_set = df[~mask]

print(training_set.shape)
print(testing_set.shape)

(1085, 7)
(253, 7)


In [4]:
# save training and test sets to be used later
training_set.to_csv("../../data/training_set.csv")
testing_set.to_csv("../../data/testing_set.csv")

In [5]:
# separating the feature columns from the target column
feature_columns = ["age", "sex", "bmi", "children", "smoker", "region"]
target_column = "charges"

X_train = training_set[feature_columns]
y_train = training_set[target_column]

X_test = testing_set[feature_columns]
y_test = testing_set[target_column]

## Apply the Preprocessing


In [6]:
# loading the preprocessing pipeline we built in the previous notebook
transformer = joblib.load("transformer.joblib")

In [7]:
# applying the column transformer
features = transformer.fit_transform(X_train)

features

array([[18.   , 33.77 ,  1.   , ...,  0.   ,  1.   ,  2.   ],
       [28.   , 33.   ,  3.   , ...,  0.   ,  1.   ,  2.   ],
       [33.   , 22.705,  0.   , ...,  0.   ,  1.   ,  1.   ],
       ...,
       [18.   , 36.85 ,  0.   , ...,  0.   ,  0.   ,  2.   ],
       [21.   , 25.8  ,  0.   , ...,  0.   ,  0.   ,  3.   ],
       [61.   , 29.07 ,  0.   , ...,  1.   ,  0.   ,  1.   ]])

## Find an Optimal Pipeline

In [8]:
tpot_regressor = TPOTRegressor(generations=50,
                               population_size=50,
                               random_state=42,
                               cv=5,
                               n_jobs=8,
                               verbosity=2,
                               early_stop=10)

In [ ]:
tpot_regressor = tpot_regressor.fit(features, y_train)

Version 0.11.7 of tpot is outdated. Version 0.12.1 was released Tuesday August 15, 2023.


Optimization Progress:   0%|          | 0/2550 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -21350273.188925236

Generation 2 - Current best internal CV score: -21177319.229764502

Generation 3 - Current best internal CV score: -21177319.229764502

Generation 4 - Current best internal CV score: -21177319.229764502

Generation 5 - Current best internal CV score: -21177319.229764502

Generation 6 - Current best internal CV score: -21139612.573396523

Generation 7 - Current best internal CV score: -21139612.573396523

Generation 8 - Current best internal CV score: -21139612.573396523

Generation 9 - Current best internal CV score: -21100308.730419137

Generation 10 - Current best internal CV score: -21062197.314579755

Generation 11 - Current best internal CV score: -21062197.314579755

Generation 12 - Current best internal CV score: -21062197.314579755

Generation 13 - Current best internal CV score: -21062197.314579755

Generation 14 - Current best internal CV score: -21057425.02162904

Generation 15 - Current best internal CV sc

## Create Pipeline

Now that we have an optimal pipeline created by TPOT we will be adding our own preprocessors to it. To do this we'll need to have an unfitted pipeline object.

To create an unfitted pipeline from the fitted pipeline that we already have, we'll clone the pipeline object:

In [ ]:
unfitted_tpot_regressor = clone(tpot_regressor.fitted_pipeline_)

unfitted_tpot_regressor

Now that we can build the same pipeline that was found by the TPOT package, we'll add our own preprocessors to the pipeline. This will ensure that the final pipeline will accept the features in the original dataset and will process the features correctly.

We'll compose the preprocessing pipeline and the tpot pipeline into one pipeline:

In [ ]:
model = Pipeline([
    ("transformer", transformer),
    ("tpot_pipeline", unfitted_tpot_regressor)
])

## Train Model

In [ ]:
model.fit(X_train, y_train)

## Test Model With Single Sample

In [ ]:
# testing the ColumnTransformer
test_df = pd.DataFrame([[65, "male", 12.5, 0, "yes", "southwest"]],
                       columns=["age", "sex", "bmi", "children", "smoker", "region"])


result = model.predict(test_df)

result

## Save Model

In [ ]:
joblib.dump(model, "model.joblib")